# 1. Import Dependencies


In [1]:
import os
import pandas as pd



# 2. Data Loading
### 2.1 Define File Paths

In [2]:
BUSINESS_PLANNING = os.path.join('data', 'CensusBusinessPlanning')
business_files = os.listdir(BUSINESS_PLANNING)

QUARTERLY_EMPLOYMENT = os.path.join('data', 'QuarterlyCensusEmployment')
quarterly_employment_files = os.listdir(QUARTERLY_EMPLOYMENT)

YEARLY_EMPLOYMENT = os.path.join('data', 'YearlyCensusEmployment')
yearly_employmet_files = os.listdir(YEARLY_EMPLOYMENT)

### 2.2 Build Data Loading Function

In [59]:
def load_files(path, dir_list):
    data_frames = {}
    
    for file in dir_list:
        var = file.split(".")[0]
        file_path = os.path.join(path, file)
        
        if '.csv' in file:
            data_frames[var] = pd.read_csv(file_path, dtype=str, low_memory=False)
        elif '.xlsx' in file:
            print("Loading and converting file:", file)
            
            # Convert Excel to CSV
            df = pd.read_excel(file_path, sheet_name=0, engine='calamine')
            
            # Create CSV file path
            csv_file_path = os.path.join(path, f"{var}.csv")
            df.to_csv(csv_file_path, index=False)
            
            print(f"Converted {file} to CSV: {var}.csv")
            
            # Store the dataframe
            data_frames[var] = df
    
    return data_frames


### 2.3 Load in Files as Dictionaries

In [4]:
cbp_dict = load_files(BUSINESS_PLANNING, business_files)

q_emp_dict = load_files(QUARTERLY_EMPLOYMENT, quarterly_employment_files)

y_emp_dict = load_files(YEARLY_EMPLOYMENT, yearly_employmet_files)


### 2.4 Load in Other Files

In [60]:
county_gdp = pd.read_csv('data/CountyGDP.csv', dtype=str, low_memory=False, encoding='latin-1')

county_income = pd.read_csv("data/CountyIncome.csv", dtype = str, low_memory = False, encoding='latin-1' )

construction_permits = pd.read_csv("Data/ResidentialConstructionPermitsCounty.csv", dtype=str, low_memory = False, encoding='latin-1') 

# 3. Data Preprocessing

### 3.1 Census Business Planning

In [61]:
def standardize_cbp_columns(cbp_dict):
    """Remove unnecessary columns and standardize based on year"""
    for df in cbp_dict.keys():
        year = df[0:4]
        if int(year) < 2017:
            use = "2012"
        else:
            use = "2017"
        keep = ['GEO_ID', 'YEAR', 'NAME', f'NAICS{use}', f'NAICS{use}_LABEL',  
                'EMPSZES_LABEL', 'ESTAB', 'EMP', 'PAYANN']
        cbp_dict[df] = cbp_dict[df][keep]
    return cbp_dict

def remove_first_row(cbp_dict):
    """Remove first row from CBP"""
    try:
        for df in cbp_dict.keys():
            cbp_dict[df] = cbp_dict[df].drop(0)
    except:
        print("Row 0 already removed")
    return cbp_dict

def combine_dfs(dict):
    standardized_df = []
    for key, value in dict.items():
        df_copy = value.copy()
        df_copy = df_copy.rename(columns={
            "NAICS2012" : "NAICS", 
            "NAICS2017" : "NAICS", 
            "NAICS2012_LABEL" : "NAICS_LABEL", 
            "NAICS2017_LABEL" : "NAICS_LABEL"
        })

        standardized_df.append(df_copy)
    combined_df = pd.concat(standardized_df, ignore_index=True)
    return combined_df

cbp_dict = remove_first_row(cbp_dict)
cbp_dict = standardize_cbp_columns(cbp_dict)
cbp_df = combine_dfs(cbp_dict)

Row 0 already removed


In [62]:
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

def filter_cbp(df):
    temp = df[df['EMPSZES_LABEL'] == "All establishments"]
    temp = temp[temp['NAICS_LABEL'].isin(['Total for all sectors', 'Retail trade', 'Accommodation and food services'])]
    return temp


# Filter for specific industries
cbp_df = filter_cbp(cbp_df)

# Seperate County and State from NAME
cbp_df[['County', 'State']] = cbp_df['NAME'].str.split(",", expand=True)
cbp_df['State'] = cbp_df['State'].str.strip().map(state_abbrev)
cbp_df = cbp_df.dropna(subset=['State'])
cbp_df = cbp_df.reset_index(drop=True)
cbp_df['County'] = cbp_df['County'].str.split().str[:-1].str.join(' ').str.upper()

## 3.2 Quarterly Employment

In [63]:
def process_quarterly_employment_data(q_emp_dict, state_abbrev):
   """
   Process all employment dataframes in dictionary and combine into one.
   
   Parameters:
   q_emp_dict: Dictionary with dataframe names as keys and dataframes as values
   state_abbrev: Dictionary mapping state names to abbreviations
   
   Returns:
   Combined processed dataframe
   """
   processed_dfs = []
   
   for key, df in q_emp_dict.items():
       temp = df.copy()
       temp = temp.dropna(subset=['St Name'])
       
       
       numeric_cols = temp.iloc[:, 14:17].apply(pd.to_numeric, errors='coerce')
       temp.loc[:, 'average_employment'] = numeric_cols.mean(axis=1)
       
       # Drop original columns and filter for counties
       temp = temp.drop(columns=temp.columns[14:17]) 
       temp = temp[temp["Area Type"] == "County"]
       
       # Split area into county and state
       temp[['County', 'State']] = temp['Area'].str.split(",", expand=True)
       temp['State'] = temp['State'].str.strip().map(state_abbrev)
       
       # Select and reorder columns
       temp = temp[['Year', 'Qtr','County', 'State','NAICS', 
                  'Area Type', 'Ownership', 'Industry', 'Status Code',
                  'Establishment Count', 'Total Quarterly Wages', 'Average Weekly Wage',
                  'Employment Location Quotient Relative to U.S.',
                  'Total Wage Location Quotient Relative to U.S.', 'average_employment']]
       
       processed_dfs.append(temp)
   
   # Combine all processed dataframes
   combined_df = pd.concat(processed_dfs, ignore_index=True)
   
   return combined_df


quarterly_employment_df = process_quarterly_employment_data(q_emp_dict, state_abbrev)
quarterly_employment_df['County'] = quarterly_employment_df['County'].str.split().str[:-1].str.join(' ').str.upper()

## 3.3 Yearly Employment

In [64]:
def process_yearly_employment_data(y_emp_dict, state_abbrev):
   """
   Process all employment dataframes in dictionary and combine into one.
   
   Parameters:
   q_emp_dict: Dictionary with dataframe names as keys and dataframes as values
   state_abbrev: Dictionary mapping state names to abbreviations
   
   Returns:
   Combined processed dataframe
   """
   processed_dfs = []
   
   for key, df in y_emp_dict.items():
       temp = df.copy()
       temp = temp[temp["Area Type"] == "County"]
       
       # Split area into county and state
       temp[['County', 'State']] = temp['Area'].str.split(",", expand=True)
       temp['State'] = temp['State'].str.strip().map(state_abbrev)
       
       processed_dfs.append(temp)
   
   
   combined_df = pd.concat(processed_dfs, ignore_index=True)
   combined_df = combined_df[[ 'Year', 'Qtr', 'County', 'State','NAICS',
       'Area Type', 'Ownership', 'Industry',
       'Annual Average Status Code', 'Annual Average Establishment Count',
       'Annual Average Employment', 'Annual Total Wages',
       'Annual Average Weekly Wage', 'Annual Average Pay',
       'Employment Location Quotient Relative to U.S.',
       'Total Wage Location Quotient Relative to U.S.', ]]
   
   
   return combined_df

yearly_employment_df = process_yearly_employment_data(y_emp_dict, state_abbrev)
yearly_employment_df['County'] = yearly_employment_df['County'].str.split().str[:-1].str.join(' ').str.upper()


## 3.4 County Income and GDP

In [65]:
def process_county(county_df):
   """
   Process county dataframes by splitting GeoName into County and State,
   cleaning the data, and reordering columns.
   
   Parameters:
   df_df: DataFrame with county income data
   
   Returns:
   Processed DataFrame with cleaned County and State columns
   """
   df = county_df[['GeoFIPS', 'GeoName', 'TableName',
                                    'IndustryClassification', 'Description', 'Unit', '2014', '2015', '2016',
                                    '2017', '2018', '2019', '2020', '2021', '2022', '2023']].copy()
   
   # Filter to only rows that contain a comma
   df = df[df['GeoName'].str.contains(',', na=False)]
   
   # Split GeoName into County and State
   df[['County', 'State']] = df['GeoName'].str.split(",", n=1, expand=True)
   
   # Clean County and State columns
   df['County'] = df['County'].str.strip()
   df['State'] = df['State'].str.strip().str.replace('*', '', regex=False)
   df['County'] = df['County'] + " County"
   
   df = df[['GeoFIPS', 'GeoName', 'County', 'State',
                                 'Description', 'Unit', '2014', '2015', '2016',
                                 '2017', '2018', '2019', '2020', '2021', '2022', '2023']]
   
   return df

processed_income = process_county(county_income)
processed_income['County'] = processed_income['County'].str.split().str[:-1].str.join(' ').str.upper()
processed_gdp = process_county(county_gdp)
processed_gdp['County'] = processed_gdp['County'].str.split().str[:-1].str.join(' ').str.upper()



## 3.5 Construction

In [ ]:
import re

def prepare_permits(df, start_year=2014, end_year=2022):
    """
    Prepare permits data specifically for residential development impact forecasting
    
    Parameters:
    df: DataFrame with building permits data
    start_year: Starting year for analysis (default: 2014)
    end_year: Ending year for analysis (default: 2022)
    """
    
    # Base identification columns
    base_cols = ['GEOID', 'STATE', 'COUNTY', 'NAME', 'STATE_NAME']
    
    # Validate required columns exist
    missing_cols = [col for col in base_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    surge_cols = []
   
    for year in range(start_year, end_year + 1):
        if year == 2022:
            # Special case for 2022 with different column name
            col_name = f'MULTIFAMILY_PERMITS_5_OR_MORE_{year}'
        else:
            col_name = f'MULTIFAMILY_PERMITS_5_OR_MORE_UNITS_{year}'
        
        if col_name in df.columns:
            surge_cols.append(col_name)
        else:
            print(f"Warning: Column {col_name} not found in dataset")
    
    if not surge_cols:
        raise ValueError(f"No 5+ unit permit columns found for years {start_year}-{end_year}")
        
    project_df = df[base_cols + surge_cols].copy()
    
    # Reshape to long format for time series analysis
    project_long = project_df.melt(
        id_vars=base_cols,
        value_vars=surge_cols,
        var_name='year_col',
        value_name='permits_5plus'
    )
    
    project_long['year'] = project_long['year_col'].str.extract(r'(\d{4})').astype(int)
    project_long = project_long.drop('year_col', axis=1)
    
    # Handle missing values and ensure proper data types
    project_long['permits_5plus'] = pd.to_numeric(project_long['permits_5plus'], errors='coerce').fillna(0)
    
    # Sort by location and year for better organization
    project_long = project_long.sort_values(['STATE_NAME', 'NAME', 'year']).reset_index(drop=True)
    return project_long

def fix_permits(df, state_abbrev):
    df = df[['NAME','STATE_NAME', 'year', 'permits_5plus']].copy()
    df['NAME'] = df['NAME'].str.upper()
    df['STATE_NAME'] = df['STATE_NAME'].str.strip().map(state_abbrev)

    df = df.rename(columns={
    "NAME" : "County", 
    "STATE_NAME" : "State"
    })
    return df

const_permits = prepare_permits(construction_permits)
const_permits = fix_permits(const_permits, state_abbrev)

,County,State,year,permits_5plus
0,AUTAUGA,AL,2014,0.0
1,AUTAUGA,AL,2015,0.0
2,AUTAUGA,AL,2016,0.0
3,AUTAUGA,AL,2017,0.0
4,AUTAUGA,AL,2018,0.0
...,...,...,...,...
28984,WESTON,WY,2018,0.0
28985,WESTON,WY,2019,0.0
28986,WESTON,WY,2020,0.0
28987,WESTON,WY,2021,0.0


# 4. Data Exploration

## 4.1 Census Business Planning

In [67]:
# Convert string columns to numeric
numeric_cols = ['YEAR', 'ESTAB', 'EMP', 'PAYANN']

for col in numeric_cols:
    cbp_df[col] = pd.to_numeric(cbp_df[col], errors='coerce')

    # Check the conversion worked
print("Data types after conversion:")
print(cbp_df[numeric_cols].dtypes)
print("\nAny missing values created?")
print(cbp_df[numeric_cols].isnull().sum())

Data types after conversion:
YEAR        int64
ESTAB     float64
EMP       float64
PAYANN    float64
dtype: object

Any missing values created?
YEAR        0
ESTAB       1
EMP       652
PAYANN    433
dtype: int64


#### Functions to create visuals 

In [ ]:
import pandas as pd
from IPython.display import display

def get_sector_summary(results):
    summary_data = []
    sector_mapping = {
        'all_sectors': 'All Sectors',
        'retail_trade': 'Retail Trade',
        'accommodation_food': 'Accommodation & Food Services'
    }
    
    for key, sector_name in sector_mapping.items():
        if results[key] is not None and not results[key].empty:
            data = results[key]
            summary_data.append({
                'Sector': sector_name,
                'Counties_Analyzed': len(data),
                'Max_Employment': data['EMP'].max(),
                'Min_Employment': data['EMP'].min(),
                'Avg_Employment': data['EMP'].mean(),
                'Top_County': f"{data.iloc[0]['County']}, {data.iloc[0]['State']}"
            })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

def plot_top_counties(results, sector='all_sectors', top_n=10):  
    import matplotlib.pyplot as plt
    
    if results[sector] is None or results[sector].empty:
        print(f"No data available for sector: {sector}")
        return
    
    data = results[sector].head(top_n)
    
    # Create labels with county and state
    labels = [f"{row['County']}, {row['State']}" for _, row in data.iterrows()]
    
    # Create the plot
    plt.figure(figsize=(8, 6))
    bars = plt.barh(range(len(data)), data['EMP'], color='steelblue', alpha=0.7)
    
    # Customize the plot
    plt.yticks(range(len(data)), labels)
    plt.xlabel('Average Employment')
    plt.title(f'Top {top_n} Counties by Average Employment - {sector.replace("_", " ").title()}')
    plt.gca().invert_yaxis()  # Highest values at the top
    
    # Add value labels on bars
    for i, (bar, value) in enumerate(zip(bars, data['EMP'])):
        plt.text(bar.get_width() + max(data['EMP']) * 0.01, 
                bar.get_y() + bar.get_height()/2, 
                f'{value:,.0f}', 
                ha='left', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

def get_employment_summary_stats(cbp_df, sectors=['Total for all sectors', 'Retail trade', 'Accommodation and food services']):
    """
    Get summary statistics for employment distributions across different sectors using ALL data.
    """
    
    # Group data (same as in analyze_county_employment but use ALL data)
    cbp_group_county = cbp_df.groupby(['County', 'State', 'NAICS_LABEL'])['EMP'].mean().reset_index()
    
    summary_data = []
    
    for sector in sectors:
        sector_data = cbp_group_county[cbp_group_county['NAICS_LABEL'] == sector]
        
        if not sector_data.empty:
            # Remove any NaN or infinite values
            clean_data = sector_data['EMP'].dropna()
            clean_data = clean_data[clean_data != float('inf')]
            clean_data = clean_data[clean_data != float('-inf')]
            
            if len(clean_data) > 0:
                summary_data.append({
                    'Sector': sector,
                    'Count': len(clean_data),
                    'Mean': clean_data.mean(),
                    'Median': clean_data.median(),
                    'Std': clean_data.std(),
                    'Min': clean_data.min(),
                    'Max': clean_data.max(),
                    'Q1': clean_data.quantile(0.25),
                    'Q3': clean_data.quantile(0.75)
                })
            else:
                print(f"No valid data for sector: {sector}")
        else:
            print(f"No data found for sector: {sector}")
    
    if summary_data:
        return pd.DataFrame(summary_data)
    else:
        print("No data available for any of the specified sectors")
        return pd.DataFrame()


In [ ]:
def filter_down_counties(df, top_n=500):
    # Group by county with average employment since 2014
    df_group_county = df.groupby(['County', 'State', 'NAICS_LABEL'])['EMP'].mean().reset_index()

    # Filter to all sectors and keep top n
    all_sectors = df_group_county[df_group_county['NAICS_LABEL'] == "Total for all sectors"]
    all_sectors = all_sectors.nlargest(top_n, 'EMP')

    # Get list of top n counties
    counties = all_sectors['County'].to_list()
    states = all_sectors['State'].to_list()

    top_county_state = set(zip(counties, states))

    # Filter Retail Trade
    retail_trade = df_group_county[
        (df_group_county['NAICS_LABEL'] == "Retail trade") &
        (df_group_county.apply(lambda row: (row['County'], row['State']) in top_county_state, axis=1))
    ].sort_values("EMP", ascending=False)

    # Filter Accommodation and Food Services
    accommodation_food = df_group_county[
        (df_group_county['NAICS_LABEL'] == "Accommodation and food services") &
        (df_group_county.apply(lambda row: (row['County'], row['State']) in top_county_state, axis=1))
    ].sort_values("EMP", ascending=False)

    print("Retail Trade sample:")
    print(retail_trade.head())

    print("Accommodation and Food Services sample:")
    print(accommodation_food.head())

    results = {
        'grouped_data': df_group_county,
        'all_sectors': all_sectors,
        'retail_trade': retail_trade,
        'accommodation_food': accommodation_food
    }
    return results


In [ ]:
results = filter_down_counties(cbp_df, top_n=500)

# Get summary statistics
summary = get_sector_summary(results)
print("\nEmployment Analysis Summary")
print("=" * 30)
display(summary)

In [ ]:
# Plot results - these use the top N from results
plot_top_counties(results, sector='all_sectors', top_n=10)
plot_top_counties(results, sector='retail_trade', top_n=10)
plot_top_counties(results, sector='accommodation_food', top_n=10)

In [92]:
# Get summary statistics for ALL counties in each sector
full_stats = get_employment_summary_stats(cbp_df)
print("\nFull Employment Statistics (All Counties)")
print("=" * 40)
display(full_stats)


Full Employment Statistics (All Counties)


,Sector,Count,Mean,Median,Std,Min,Max,Q1,Q3
0,Total for all sectors,3148,39623.299975,6579.65,145142.824679,21.000000,3883290.7,2143.425000,20917.050
1,Retail trade,3138,5035.436825,1013.55,15433.634038,5.666667,416644.2,323.175000,3255.425
2,Accommodation and food services,3112,4429.377898,741.15,16083.290330,1.000000,436935.9,214.361111,2597.175


In [93]:
#  Generate lists to focus on for analysis

reducedCounties = results['all_sectors']['County'].tolist()
reducedStates = results['all_sectors']['State'].tolist()
reducedFull = list(zip(reducedCounties, reducedStates))


## 4.2 Quarterly Employment

In [171]:
def filter_employment(df, reducedFull):
    df = df.copy()
    filter_index = pd.MultiIndex.from_tuples(reducedFull, names=['County', 'State'])
    filtered_df = df[df.set_index(['County', 'State']).index.isin(filter_index)].reset_index(drop=True)
    return filtered_df


quarterly_employment_df = filter_employment(quarterly_employment_df, reducedFull)
yearly_employment_df = filter_employment(yearly_employment_df, reducedFull)
processed_income = filter_employment(processed_income, reducedFull)
processed_gdp = filter_employment(processed_gdp, reducedFull)
const_permits = filter_employment(const_permits, reducedFull)

In [172]:
# Create (county, state) tuple sets for comparison
reducedFull_tuples = set(reducedFull)

datasets = {
    'quarterly employment': set(zip(quarterly_employment_df['County'], quarterly_employment_df['State'])),
    'yearly employment': set(zip(yearly_employment_df['County'], yearly_employment_df['State'])),
    'processed income': set(zip(processed_income['County'], processed_income['State'])),
    'processed gdp': set(zip(processed_gdp['County'], processed_gdp['State'])), 
    'construction permits': set(zip(const_permits['County'], const_permits['State']))
}

# Compare CBP with each dataset
for name, dataset_tuples in datasets.items():
    cbp_only = reducedFull_tuples - dataset_tuples
    dataset_only = dataset_tuples - reducedFull_tuples
    
    print(f"\n{name.title()}:")
    print(f"  CBP only: {len(cbp_only)} counties")
    print(f"  {name} only: {len(dataset_only)} counties")
    
    if cbp_only:
        print(f"  CBP examples: {list(cbp_only)[:3]}")
    if dataset_only:
        print(f"  {name} examples: {list(dataset_only)[:3]}")


Quarterly Employment:
  CBP only: 20 counties
  quarterly employment only: 0 counties
  CBP examples: [('NORFOLK', 'VA'), ('LYNCHBURG', 'VA'), ('FAIRFAX', 'VA')]

Yearly Employment:
  CBP only: 20 counties
  yearly employment only: 0 counties
  CBP examples: [('NORFOLK', 'VA'), ('LYNCHBURG', 'VA'), ('FAIRFAX', 'VA')]

Processed Income:
  CBP only: 20 counties
  processed income only: 0 counties
  CBP examples: [('NORFOLK', 'VA'), ('LYNCHBURG', 'VA'), ('FAIRFAX', 'VA')]

Processed Gdp:
  CBP only: 20 counties
  processed gdp only: 0 counties
  CBP examples: [('NORFOLK', 'VA'), ('LYNCHBURG', 'VA'), ('FAIRFAX', 'VA')]

Construction Permits:
  CBP only: 9 counties
  construction permits only: 0 counties
  CBP examples: [('CAPITOL PLANNING', 'CT'), ('WESTERN CONNECTICUT PLANNING', 'CT'), ('DISTRICT OF', 'DC')]


In [173]:
# Create (county, state) tuple sets for all datasets
reducedFull_tuples = set(reducedFull)
quarterly_tuples = set(zip(quarterly_employment_df['County'], quarterly_employment_df['State']))
yearly_tuples = set(zip(yearly_employment_df['County'], yearly_employment_df['State']))
income_tuples = set(zip(processed_income['County'], processed_income['State']))
gdp_tuples = set(zip(processed_gdp['County'], processed_gdp['State']))
const_tuples = set(zip(const_permits['County'], const_permits['State']))

# Find intersection of all county-state tuple sets
common_county_state_tuples = reducedFull_tuples.intersection(
    quarterly_tuples,
    yearly_tuples,
    income_tuples,
    gdp_tuples, 
    const_tuples
)

# Convert back to list if needed
common_county_state_tuples = list(common_county_state_tuples)

print(f"Number of counties common to all datasets: {len(common_county_state_tuples)}")
print(f"Common counties (first 10): {common_county_state_tuples[:10]}")

Number of counties common to all datasets: 480
Common counties (first 10): [('OLMSTED', 'MN'), ('GRAND TRAVERSE', 'MI'), ('WINNEBAGO', 'IL'), ('POLK', 'FL'), ('ORANGE', 'NC'), ('OKALOOSA', 'FL'), ('CHESTERFIELD', 'VA'), ('CLARK', 'NV'), ('SARASOTA', 'FL'), ('ESSEX', 'MA')]


In [174]:
# More filtering based on common counties
quarterly_employment_df = filter_employment(quarterly_employment_df, common_county_state_tuples)
yearly_employment_df = filter_employment(yearly_employment_df, common_county_state_tuples)
processed_income = filter_employment(processed_income, common_county_state_tuples)
processed_gdp = filter_employment(processed_gdp, common_county_state_tuples)
const_permits = filter_employment(const_permits, common_county_state_tuples)

In [175]:
from collections import Counter
states = [state for (county, state) in common_county_state_tuples]
counts = Counter(states)
counts

Counter({'TX': 32,
         'CA': 31,
         'FL': 28,
         'PA': 28,
         'NY': 24,
         'OH': 21,
         'NC': 18,
         'NJ': 17,
         'MI': 16,
         'GA': 16,
         'IL': 15,
         'WI': 14,
         'IN': 14,
         'TN': 13,
         'CO': 12,
         'SC': 12,
         'MA': 11,
         'WA': 10,
         'AL': 10,
         'LA': 10,
         'MN': 9,
         'MD': 9,
         'MO': 8,
         'IA': 7,
         'OR': 7,
         'AZ': 7,
         'MS': 7,
         'VA': 6,
         'UT': 6,
         'CT': 6,
         'ME': 5,
         'KY': 5,
         'AR': 5,
         'KS': 4,
         'NH': 4,
         'ID': 4,
         'WV': 3,
         'DE': 3,
         'RI': 3,
         'OK': 3,
         'MT': 3,
         'NE': 3,
         'NV': 2,
         'HI': 2,
         'SD': 2,
         'NM': 2,
         'ND': 2,
         'VT': 1})

# FIPS Code Merging

# Development Surge Indicator Creation

# Quarterly Frequency Standardization

# Feature Engineering (PDFM + Economic Variables)

# Train/Test Split

# TimesFM Model Setup

# Baseline Forecasting

# Treatment Effect Analysis

# Model Validation & Backtesting

# Insights & Visualization

# Dashboard Prep